imports

In [21]:
import torch
import os
import pandas as pd
import numpy as np

from HGNN.train.configParser import ConfigParser
from HGNN.train import CNN, dataLoader

experimentsFileName = "experiments.csv"

parameters

In [22]:
experimentsPath="/home/jnl47/Documents/dev/HGNN/experiments/"
dataPath="/home/jnl47/Documents/dev/HGNN/data"
experimentName="smegma"
trial_hash="06feff02666707cbdecf12fdbec650a36bef56a47085a37a2ddf1515"

cuda=1

cuda

In [23]:
# set cuda
if torch.cuda.is_available():
    torch.cuda.set_device(cuda)
    print("using cuda", cuda)

using cuda 1


Get dataset

In [24]:
experimentsFileNameAndPath = os.path.join(experimentsPath, experimentsFileName)
if os.path.exists(experimentsFileNameAndPath):
    experiments_df = pd.read_csv(experimentsFileNameAndPath)
experiments_df["trialHash"]
experiments_df[experiments_df["trialHash"] == trial_hash]

,experimentName,modelName,datasetName,experimentHash,trialHash,image_path,suffix,training_count,validation_count,batchSize,...,learning_rate,numOfTrials,patience,fc_width,fc_layers,modelType,lambda,unsupervisedOnTest,tl_model,augmented
2,smegma,models/06feff02666707cbdecf12fdbec650a36bef56a...,datasplits/0baedd7ad9e196e933c0628cd848466e306...,11cd54ba032786ffb6dda3320bd9466e9c5a8ee5903a0a...,06feff02666707cbdecf12fdbec650a36bef56a47085a3...,INHS_segmented_padded_fish,52,0.64,0.16,32,...,0.0001,5,50,200,1,BB,0.6,False,ResNet18,False


In [25]:
# Get experiment parameters
experimentsFileNameAndPath = os.path.join(experimentsPath, experimentsFileName)
if os.path.exists(experimentsFileNameAndPath):
    experiments_df = pd.read_csv(experimentsFileNameAndPath)
else:
    raise Exception("Experiment not " + trial_hash + " found!")
experimentRecord = experiments_df[experiments_df["trialHash"] == trial_hash]
experiment_params = experimentRecord.to_dict('records')[0]
print(experiment_params)

config_parser = ConfigParser(experimentsPath, dataPath, experimentName)
experimentPathAndName = os.path.join(experimentsPath, experimentName)
datasetManager = dataLoader.datasetManager(experimentPathAndName)
datasetManager.updateParams(config_parser.fixPaths(experiment_params))
dataset = datasetManager.getDataset()
train_loader, validation_loader, test_loader = datasetManager.getLoaders()

Loading images:   0%|          | 0/550 [00:00<?, ?it/s, fileName=INHS_FISH_68432.jpg]

{'experimentName': 'smegma', 'modelName': 'models/06feff02666707cbdecf12fdbec650a36bef56a47085a37a2ddf1515', 'datasetName': 'datasplits/0baedd7ad9e196e933c0628cd848466e3063d51e2319593293cab55c', 'experimentHash': '11cd54ba032786ffb6dda3320bd9466e9c5a8ee5903a0a9f7957c81b', 'trialHash': '06feff02666707cbdecf12fdbec650a36bef56a47085a37a2ddf1515', 'image_path': 'INHS_segmented_padded_fish', 'suffix': 52, 'training_count': 0.64, 'validation_count': 0.16, 'batchSize': 32, 'n_epochs': 5000, 'learning_rate': 0.0001, 'numOfTrials': 5, 'patience': 50, 'fc_width': 200, 'fc_layers': 1, 'modelType': 'BB', 'lambda': 0.6, 'unsupervisedOnTest': False, 'tl_model': 'ResNet18', 'augmented': False}
Creating dataset...


Transforming images:   2%|▏         | 9/550 [00:00<00:06, 85.93it/s]

Creating dataset... Done.


Transforming images: 100%|██████████| 550/550 [00:09<00:00, 60.88it/s]


train/test =  440 110
train/val =  352 88
Creating loaders...
Creating loaders... Done.
file /home/jnl47/Documents/dev/HGNN/experiments/smegma/datasplits/31a44652b8bc8fc1bea842ba67230348eefb450960e07f829f118faa/trainingIndex.csv written
file /home/jnl47/Documents/dev/HGNN/experiments/smegma/datasplits/31a44652b8bc8fc1bea842ba67230348eefb450960e07f829f118faa/valIndex.csv written
file /home/jnl47/Documents/dev/HGNN/experiments/smegma/datasplits/31a44652b8bc8fc1bea842ba67230348eefb450960e07f829f118faa/testIndex.csv written


Get untrained model

In [26]:
architecture = {
    "fine": len(dataset.csv_processor.getFineList()),
    "coarse" : len(dataset.csv_processor.getCoarseList())
}
dataset.csv_processor.getFineList()
model = CNN.create_model(architecture, experiment_params)
model
# get the model and the parameters
modelName = experimentRecord.iloc[0]["modelName"]
trialName = os.path.join(experimentPathAndName, modelName)
_ = CNN.loadModel(model, trialName)

In [27]:
dataset.csv_processor.getFineList()

['Lepomis auritus',
 'Noturus eleutherus',
 'Carassius auratus',
 'Lepomis cyanellus',
 'Esox americanus',
 'Noturus insignis',
 'Noturus flavus',
 'Noturus gyrinus',
 'Esox lucius',
 'Noturus exilis',
 'Cyprinus carpio']

In [28]:
model

CNN_hierarchy(
  (pretrained_model): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=

In [29]:
modelName

'models/06feff02666707cbdecf12fdbec650a36bef56a47085a37a2ddf1515'

sort through predictions

In [30]:
df_misclassified = pd.DataFrame(columns=['file name', 'true label', 'probability of true label', 'predicted label'])
df_correctlyclassified = pd.DataFrame(columns=['file name', 'true label', 'probability of true label', 'predicted label'])

# get probability of correct prediction and true label
predProblist, lbllist = CNN.getLoaderPredictionProbabilities(test_loader, model, experiment_params)
_, predlist = torch.max(predProblist, 1)
lbllist = lbllist.reshape(lbllist.shape[0], -1)
predProblist = predProblist.gather(1, lbllist)
predProblist = predProblist.reshape(1, -1)
predProblist = predProblist[0]

# sort through
predProblist, indices = torch.sort(predProblist)
predlist = predlist[indices]
lbllist = lbllist[indices]

for i, lbl in enumerate(lbllist):
    prd = predlist[i]
    prdProb = predProblist[i]
    
    if torch.cuda.is_available():
        lbl = lbl.cpu()
        prd = prd.cpu()
        prdProb = prdProb.cpu()

    s = dataset[i]
    row = {'file name' : s['fileName'] , 
           'true label' : int(lbl.numpy()), 
           'probability of true label': float(prdProb.numpy()),
           'predicted label' : int(prd.numpy())}
    
    if(lbl != prd):
        df_misclassified = df_misclassified.append(row, ignore_index=True)
    else:
        df_correctlyclassified = df_correctlyclassified.append(row, ignore_index=True)
        
df_misclassified = df_misclassified.sort_values(by=[ 'true label', 'probability of true label'])
df_correctlyclassified = df_correctlyclassified.sort_values(by=['true label', 'probability of true label'])

Display and save mispredicted

In [31]:
print(df_misclassified)
df_misclassified.to_csv(os.path.join(experimentPathAndName, modelName, 'misclassified examples.csv'))

             file name true label  probability of true label predicted label
0  INHS_FISH_68432.jpg          0                   0.029481               3
1   INHS_FISH_2426.jpg          3                   0.032985              10
4  INHS_FISH_95243.jpg          4                   0.179742               8
2  INHS_FISH_93450.jpg          6                   0.053474               9
3  INHS_FISH_89331.jpg          6                   0.057270               1


Save and display correctly predicted

Filename of correctly classified

In [32]:
print(df_correctlyclassified)
df_correctlyclassified.to_csv(os.path.join(experimentPathAndName, modelName, 'correctly classified examples.csv'))

                file name true label  probability of true label  \
13    INHS_FISH_14454.jpg          0                   0.658075   
15    INHS_FISH_76861.jpg          0                   0.693080   
27    INHS_FISH_37733.jpg          0                   0.918069   
29    INHS_FISH_41171.jpg          0                   0.933658   
32   INHS_FISH_108140.jpg          0                   0.963959   
..                    ...        ...                        ...   
85    INHS_FISH_27267.jpg         10                   0.979372   
97    INHS_FISH_33588.jpg         10                   0.981626   
98    INHS_FISH_85801.jpg         10                   0.981755   
101   INHS_FISH_83473.jpg         10                   0.983593   
104  INHS_FISH_107230.jpg         10                   0.986648   

    predicted label  
13                0  
15                0  
27                0  
29                0  
32                0  
..              ...  
85               10  
97               10

In [33]:
test_loader